In [1]:
import sys
import torch
pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
version_str="".join([
    f"py3{sys.version_info.minor}_cu",
    torch.version.cuda.replace(".",""),
    f"_pyt{pyt_version_str}"
])

In [2]:
version_str

'py39_cu113_pyt1101'

In [ ]:
!pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html

In [2]:
import numpy as np

In [3]:
np.tile(np.eye(4), (24, 1, 1)).shape

(24, 4, 4)

In [7]:
neg = np.eye(3)

neg = np.asarray([
    [1., 0., 0.],
    [0., 0., -1.],
    [0., -1., 0.]
])
neg

array([[ 1.,  0.,  0.],
       [ 0.,  0., -1.],
       [ 0., -1.,  0.]])

In [8]:
neg4 = np.asarray([
            [1., 0., 0., 0.],
            [0., 0., -1., 0.],
            [0., -1., 0., 0.],
            [0., 0., 0., 1.]
        ])
neg4

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.],
       [ 0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  1.]])

In [17]:
fake_pos = np.random.rand(24,4,4)
fake_pos_neg = np.matmul(neg4, fake_pos)
fake_pos.shape, fake_pos_neg.shape
fake_pos[10], fake_pos_neg[10]

(array([[0.81824953, 0.99023965, 0.29678433, 0.78764167],
        [0.10489509, 0.42289297, 0.11288829, 0.26049807],
        [0.42620673, 0.55940959, 0.84393757, 0.73088331],
        [0.25859443, 0.20628784, 0.8393375 , 0.31937397]]),
 array([[ 0.81824953,  0.99023965,  0.29678433,  0.78764167],
        [-0.42620673, -0.55940959, -0.84393757, -0.73088331],
        [-0.10489509, -0.42289297, -0.11288829, -0.26049807],
        [ 0.25859443,  0.20628784,  0.8393375 ,  0.31937397]]))

In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from im2mesh.utils.utils_cam import rotate_camera_by_frame_idx
import numpy as np
import os
from im2mesh.utils.utils import save_verts


In [42]:
R = np.random.rand(3,3)
T = np.random.rand(3)
R, T

# torch version
cam_rot = torch.tensor(R)
cam_trans = torch.tensor(T)
original_pose = torch.cat([F.pad(cam_rot, pad=(0, 0, 0, 1)),
                  F.pad(cam_trans, pad=(0, 1), value=1).unsqueeze(-1)],
                 dim=-1)   

# numpy version
original_pose_np = np.concatenate((
    np.pad(R, pad_width=((0, 1),(0, 0))),
    np.pad(T, pad_width=(0, 1), constant_values=1)[...,None]
), axis=-1)

torch.all(original_pose==torch.tensor(original_pose_np))
# original_pose, original_pose_np


tensor(True)

In [55]:


idx = 0
original_pose_np, np.linalg.inv(original_pose_np)
R, cam_trans = rotate_camera_by_frame_idx(original_pose_np ,idx, trans=np.asarray([0,0,0]), rotate_axis='y', inv_angle=True)
R.dtype, type(cam_trans)

(dtype('float64'), numpy.ndarray)

In [16]:
import numpy as np
import os
outfile = os.path.join('minimal_shape','cam_locs.npy')

for i in range(3):
    with open(outfile,'ab') as f:
        cam_loc = np.random.rand(3)
        print(cam_loc)
        np.save(f, cam_loc)


[0.71749851 0.91954951 0.95548599]
[0.02313291 0.90134679 0.98832069]
[0.42510703 0.41307831 0.35624575]


In [19]:
all_cams = []
try:
    with open(outfile, 'rb') as f:
        while(True):
            # a = np.load(f)
            # print(np.load(f))
            all_cams.append(np.load(f))
except:
    print('finish loading')
    print(np.stack(all_cams).shape)
# a = np.fromfile(outfile)
# # a = np.load(outfile)
# a

finish loading
(6, 3)


In [2]:

outfile = os.path.join('minimal_shape','original_cam_locs.npy')
all_cams = []
try:
    with open(outfile, 'rb') as f:
        while(True):
            all_cams.append(np.load(f))
except: 
    print('finish loading')
    print(np.stack(all_cams).shape)
    save_verts(all_cams, os.path.join('minimal_shape', 'all_cam_locs'), 3)
    exit(0)

finish loading
(210, 3)


: 

In [2]:
original_R = np.array([[-0.99369407,  0.09369569, -0.06158913],
 [-0.08569901, -0.28045344,  0.9560343 ],
 [ 0.07230341,  0.9552837,   0.28671455]])
original_T = np.array([0.05873722, 0.99344826, 2.98926   ])
# original_R.shape, original_T.shape
original_pose_np = np.concatenate((
    np.pad(original_R, pad_width=((0, 1),(0, 0))),
    np.pad(original_T, pad_width=(0, 1), constant_values=1)[...,None]
), axis=-1)
original_extrinsics = np.linalg.inv(original_pose_np)
# trans_tmp = np.asarray( [ 0.53449845,  0.5915778,  -0.9506396 ])#######################################################
# trans_tmp = np.asarray( [ 0.,  0.,  0. ])
trans_tmp = np.asarray( [ 0.53449845,  0.5915778,  -0.9506396 ])
outfile = os.path.join('minimal_shape','original_cam_locs.npy')

In [3]:
outfile = os.path.join('minimal_shape','original_cam_locs.npy')
if os.path.exists(outfile):
    os.remove(outfile)
with open(outfile,'ab') as f:
    for idx in range(210):
        R, cam_trans, _ = rotate_camera_by_frame_idx(original_extrinsics ,idx, trans=trans_tmp, rotate_axis='z', inv_angle=True)
        R, cam_trans = R.astype(np.float32), cam_trans.astype(np.float32)
        cam_loc = np.dot(-R.T, cam_trans)
        np.save(f, cam_loc)

all_cams = []
try:
    with open(outfile, 'rb') as f:
        while(True):
            all_cams.append(np.load(f))
except:
    print('finish loading')
    print(np.stack(all_cams).shape)
    save_verts(all_cams, './', 'cam_sphere_noise')


target_location before chat [ 0.53449845  0.5915778  -0.9506396 ]
Radius:3.9888165503908404
cam location [[ 0.54805856  0.5915778  -4.9394331 ]]
target_location before chat [ 0.53449845  0.5915778  -0.9506396 ]
Radius:3.9888165503908404
cam location [[ 1.05510457  0.78106293 -4.9007943 ]]
target_location before chat [ 0.53449845  0.5915778  -0.9506396 ]
Radius:3.9888165503908404
cam location [[ 0.67673013  0.71092435 -4.93513254]]
target_location before chat [ 0.53449845  0.5915778  -0.9506396 ]
Radius:3.9888165503908404
cam location [[ 0.25265007  0.10340209 -4.89942466]]
target_location before chat [ 0.53449845  0.5915778  -0.9506396 ]
Radius:3.9888165503908404
cam location [[ 0.53805479  0.61174682 -4.93940357]]
target_location before chat [ 0.53449845  0.5915778  -0.9506396 ]
Radius:3.9888165503908404
cam location [[ 0.54080279  0.55582413 -4.93929093]]
target_location before chat [ 0.53449845  0.5915778  -0.9506396 ]
Radius:3.9888165503908404
cam location [[ 0.53584931  0.58923804

In [3]:
for idx in range(10):
    cam_lookat_ray_s = np.random.rand(3)
    cam_lookat_ray_e = np.random.rand(3)
    cam_lookat_ray = np.linspace(cam_lookat_ray_s, cam_lookat_ray_e, 10)
    if idx==0:
        all_cam_lookat_ray = []
    all_cam_lookat_ray.append(cam_lookat_ray)
all_cam_lookat_ray = np.row_stack(all_cam_lookat_ray)
all_cam_lookat_ray.shape

(100, 3)

In [5]:
radius = 3.15
for frame_idx in range(10):
    x = radius if frame_idx%3==0 else 0
    y = radius if frame_idx%3==1 else 0
    z = radius if frame_idx%3==2 else 0
    cam_location = np.column_stack((x, y, z))
    print(cam_location)

[[3.15 0.   0.  ]]
[[0.   3.15 0.  ]]
[[0.   0.   3.15]]
[[3.15 0.   0.  ]]
[[0.   3.15 0.  ]]
[[0.   0.   3.15]]
[[3.15 0.   0.  ]]
[[0.   3.15 0.  ]]
[[0.   0.   3.15]]
[[3.15 0.   0.  ]]


In [2]:
from im2mesh.utils.root_finding_utils import (
            forward_skinning,
            unnormalize_canonical_points,
            normalize_canonical_points
        )

In [ ]:

points_lbs, _ = forward_skinning(points_hat,
                                loc=loc,
                                sc_factor=sc_factor,
                                coord_min=coord_min,
                                coord_max=coord_max,
                                center=center,
                                skinning_model=self.skinning_model,
                                vol_feat=vol_feat,
                                bone_transforms=inputs['bone_transforms'],
                                return_w=False
                            )